In [33]:
### Library Imports
import pandas as pd
import yfinance as yf
import numpy as np
from scipy import stats
import datetime
import matplotlib.pyplot as plt


In [34]:
### Function to Get All Tickers
def get_tickers(url):
    
    tickers_list = []
    # Get Tickers into df
    df = pd.read_html(url)
    # Convert the df to a list
    table = df[0]  # First table contains tickers
    tickers = table['Symbol'].tolist()
    # Yahoo Finance uses '-' instead of '.' (e.g., BRK.B → BRK-B)
    tickers_list = [ticker.replace('.', '-') for ticker in tickers]

    return tickers_list

url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
full_tickers = get_tickers(url)
print(full_tickers)


['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'APO', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK-B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF-B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'COIN', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', 'DRI', 'DVA', '

In [35]:
### Function to Import Stock Data
def import_stock_data(tickers, start_date, end_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date, end_date)['Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date, end_date)['Close']
    
    # Reset index to include the Date as a column
    data = data.reset_index()

    return data

start_date = '2025-01-01'
end_date = '2025-06-01'
tickers = ['MMM', 'AOS', 'ABT', 'ABBV']
stock_data = import_stock_data(tickers, start_date, end_date)
print(stock_data.head())


/var/folders/n5/bknqp6qs37l5l4bwv9wv3bs40000gn/T/ipykernel_77464/3698191304.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data[tickers] = yf.download(tickers, start_date, end_date)['Close']
[*********************100%***********************]  4 of 4 completed

        Date         MMM         AOS        ABT        ABBV
0 2025-01-02  176.135895  112.327484  66.650047  128.434326
1 2025-01-03  177.883118  112.713669  67.907227  128.602661
2 2025-01-06  176.783752  111.931412  68.204193  129.018555
3 2025-01-07  176.224243  112.287888  67.610252  131.474380
4 2025-01-08  175.213211  113.129547  67.986420  133.217194


In [36]:
### Compute the Percentage Change
def pct_change(tickers, df):
    # Add col with daily pct change for each ticker
    for t in tickers:
        df['PCT ' + t] = df[t].pct_change()
    
    # Drop null vals
    df.dropna(subset=['PCT ' + t for t in tickers], inplace=True)

    return df

### Function Call to Compute the Percentage Change
stock_data = pct_change(tickers, stock_data)
print(stock_data.head())


        Date         MMM         AOS        ABT        ABBV   PCT MMM  \
1 2025-01-03  177.883118  112.713669  67.907227  128.602661  0.009920   
2 2025-01-06  176.783752  111.931412  68.204193  129.018555 -0.006180   
3 2025-01-07  176.224243  112.287888  67.610252  131.474380 -0.003165   
4 2025-01-08  175.213211  113.129547  67.986420  133.217194 -0.005737   
5 2025-01-10  171.944534  111.208565  66.897530  129.929596 -0.018655   

    PCT AOS   PCT ABT  PCT ABBV  
1  0.003438  0.018862  0.001311  
2 -0.006940  0.004373  0.003234  
3  0.003185 -0.008708  0.019035  
4  0.007496  0.005564  0.013256  
5 -0.016980 -0.016016 -0.024678  


In [37]:
### Generate Buy Signals
''' 
Generate a "BUY" signal if the stock loses more than 5% of its value in a single day
'''
def generate_signals(df, drop_threshold):
    """
    Generate 'BUY' signals if a stock drops more than the threshold in a day.
    """
    for t in tickers:
        df['Signal ' + t] = (df['PCT ' + t] < drop_threshold).astype(int)
        # Test to see the number of generated signals
        print(f"{'Signal ' + t}: {df['Signal ' + t].sum()} buy signals generated")

    return df

### Function Retun to Generate Buy Signals
stock_data = generate_signals(stock_data, drop_threshold = -0.05)
print(stock_data.tail())


Signal MMM: 3 buy signals generated
Signal AOS: 1 buy signals generated
Signal ABT: 1 buy signals generated
Signal ABBV: 2 buy signals generated
          Date         MMM         AOS        ABT        ABBV   PCT MMM  \
97  2025-05-23  183.259995  131.300003  67.019997  147.619995  0.003944   
98  2025-05-27  185.720001  132.940002  68.570000  149.490005  0.013424   
99  2025-05-28  183.089996  132.020004  64.230003  148.660004 -0.014161   
100 2025-05-29  185.619995  132.850006  64.730003  149.630005  0.013818   
101 2025-05-30  186.110001  133.580002  64.309998  148.350006  0.002640   

      PCT AOS   PCT ABT  PCT ABBV  Signal MMM  Signal AOS  Signal ABT  \
97  -0.001521 -0.005195 -0.009594           0           0           0   
98   0.012490  0.023127  0.012668           0           0           0   
99  -0.006920 -0.063293 -0.005552           0           0           1   
100  0.006287  0.007785  0.006525           0           0           0   
101  0.005495 -0.006489 -0.008554      

In [38]:
### Trade Simulation Function
''' 
Enter trade the day after the signal and exit after a fixed holding period (ex. 5 days)
'''
def trade_sim(ticker, df, holding_days):
    trades = []

    try:
        for i in range(len(df) - holding_days):
            # If signal is 1 on day i, enter on day i+1 and exit on day i+holding_days
            if df.iloc[i][f'Signal {ticker}'] == 1:
                entry_idx = i + 1
                exit_idx = i + holding_days

                entry_date = df.index[entry_idx]
                exit_date = df.index[exit_idx]

                entry_price = df.iloc[entry_idx][ticker]
                exit_price = df.iloc[exit_idx][ticker]

                ret = (exit_price / entry_price) - 1

                trades.append({
                    'Ticker': ticker,
                    'Entry Date': entry_date,
                    'Exit Date': exit_date,
                    'Entry Price': entry_price,
                    'Exit Price': exit_price,
                    'Return': ret
                })

        return pd.DataFrame(trades)

    except KeyError as e:
        print(f"Skipping {ticker}: missing column {e}")
        return pd.DataFrame()  # Return empty DataFrame on failure

    except Exception as e:
        print(f"Error with {ticker}: {e}")
        return pd.DataFrame()



In [39]:
### Function to combine all trade data in one df
def simulate_all_trades(tickers, df, holding_days):
    """
    Simulate trades for each ticker and combine into one DataFrame.
    Skips tickers with missing price or signal columns.
    """
    all_trades = []

    for ticker in tickers:
        try:
            trades_df = trade_sim(ticker, df, holding_days)
            if not trades_df.empty:
                all_trades.append(trades_df)
        except Exception as e:
            print(f"Skipping {ticker}: {e}")
    
    if all_trades:
        return pd.concat(all_trades, ignore_index=True)
    else:
        return pd.DataFrame(columns=['Ticker', 'Entry Date', 'Exit Date', 'Entry Price', 'Exit Price', 'Return'])
    
### Function to Simulate all Trades and store in one df
full_trades_df = simulate_all_trades(tickers, stock_data, holding_days = 14)
print(full_trades_df.tail())



  Ticker  Entry Date  Exit Date  Entry Price  Exit Price    Return
0    MMM          64         77   184.841782  186.059998  0.006591
1    MMM          66         79   178.193222  193.509995  0.085956
2    AOS          64         77   124.284294  128.850006  0.036736
3   ABBV          63         76   126.291466  138.203125  0.094319
4   ABBV          64         77   127.923477  136.650742  0.068223
